<a href="https://colab.research.google.com/github/Voyageran/StartNN/blob/main/Pytorch_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/gdrive')
sys.path.insert(0,"/content/content/notebooks/colabInstallPackage")

!cp -av '/content/gdrive/MyDrive/Colab Notebooks/d2l' '/content/'

Mounted at /content/gdrive
'/content/gdrive/MyDrive/Colab Notebooks/d2l' -> '/content/d2l'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/paddle.py' -> '/content/d2l/paddle.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/mxnet.py' -> '/content/d2l/mxnet.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/tensorflow.py' -> '/content/d2l/tensorflow.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/torch.py' -> '/content/d2l/torch.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__init__.py' -> '/content/d2l/__init__.py'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__pycache__' -> '/content/d2l/__pycache__'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__pycache__/mxnet.cpython-310.pyc' -> '/content/d2l/__pycache__/mxnet.cpython-310.pyc'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__pycache__/__init__.cpython-310.pyc' -> '/content/d2l/__pycache__/__init__.cpython-310.pyc'
'/content/gdrive/MyDrive/Colab Notebooks/d2l/__pycache__/torch.cpython-310.pyc' -> '/content/d2l/__pycache__/torch.cpython-

# **Layers and Modules**


## **Block**

块（block）可以描述单个层、由多个层组成的组件或整个模型本身。

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F #

# Simple single NN
net = nn.Sequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2,20)
net(X).shape #outputs

torch.Size([2, 10])

**自定义块**

任何一个层和一个神经网络都是Module的一个subclass。

Define a class MLP which is the subclass of `nn.Module`

****Remark:***

model == nn.Linear(2, 1)

input = torch.Tensor([1, 2])
output = model(input)

print(output)

In [ ]:
class MLP(nn.Module):
  def __init__(self):
    super().__init__() # Review inherent
    self.hidden = nn.Linear(20,256) #model==nn.Linear(20,256)
    self.out = nn.Linear(256, 10)

  def forward(self, X):
    return self.out(F.relu(self.hidden(X)))

In [ ]:
#instantiate
net = MLP()
net(X)  # == net.forward(X)

tensor([[ 0.2917,  0.0283, -0.3037,  0.1345, -0.2363, -0.0971,  0.2100,  0.1017,
          0.1113, -0.3179],
        [ 0.2103,  0.0192, -0.2455,  0.1382, -0.1039,  0.0098, -0.0248,  0.1356,
          0.0627, -0.1701]], grad_fn=<AddmmBackward0>)

In [ ]:
X.shape, net(X).shape, net.forward(X).shape

(torch.Size([2, 20]), torch.Size([2, 10]), torch.Size([2, 10]))

In [ ]:
# 试一下nnSequential
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for block in args: #add layers, as keys
      self._modules[str(block)] = block #ordered dict, insert by order

  def forward(self, X):
    for block in self._modules.values(): #values返回的list跟插入进去返回的顺序一样
      X = block(X) #values里面的keys
    return X

In [ ]:
net = MySequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256, 10))

In [ ]:
print(net.forward(X))

tensor([[-0.0211,  0.1789,  0.0505,  0.2012,  0.0369,  0.1904,  0.0514, -0.0115,
          0.1747, -0.0095],
        [-0.0809,  0.0909,  0.1927,  0.0850, -0.2080,  0.1496,  0.0335, -0.0416,
          0.2090, -0.0328]], grad_fn=<AddmmBackward0>)


In [ ]:
net = MySequential(nn.Linear(20,256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0420, -0.0817,  0.0341, -0.0889,  0.1902, -0.1443,  0.0811, -0.0778,
          0.0302,  0.2398],
        [-0.0157, -0.1873,  0.0423,  0.0459,  0.0532,  0.0789,  0.0298, -0.1673,
         -0.1093,  0.1977]], grad_fn=<AddmmBackward0>)

In [ ]:
# 展示一下我们可以在init和forward函数中可以多么爽地自定义（瞎搞）
class FixedHiddenMLP(nn.Module):
  def __init__(self): # review: self is a instance,可以直接用类调用，也可以创建实例再调用
    super().__init__()
    self.rand_weight = torch.rand((20,20), requires_grad = False) #不需要weight的梯度
    self.linear = nn.Linear(20,20)

  def forward(self, X):
    X = self.linear(X)
    X = F.relu(torch.mm(X, self.rand_weight) + 1)
    X = self.linear(X)
    while X.abs().sum()>1:
      return X.sum()

In [ ]:
net = FixedHiddenMLP()
net(X)

tensor(-0.5235, grad_fn=<SumBackward0>)

In [ ]:
#嵌套玩法
class NestMLP(nn.Module):
  def __init__(self):
    super().__init__()
    # sequential也是Module的一个subclass
    self.net = nn.Sequential(nn.Linear(20,64), nn.ReLU(),
                             nn.Linear(64,32), nn.ReLU())
    self.linear = nn.Linear(32,16)

  def forward(self, X):
    return self.linear(self.net(X))

In [ ]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16,20), FixedHiddenMLP())
chimera(X)

tensor(-3.2638, grad_fn=<SumBackward0>)

## **Visiting Parameters**

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F #

# Simple single NN
net = nn.Sequential(nn.Linear(4,8), nn.ReLU(), nn.Linear(8, 1))

X = torch.rand(size = (2,4))
net(X)

tensor([[-0.2717],
        [-0.2763]], grad_fn=<AddmmBackward0>)

In [ ]:
# test nn.Linear(8, 1)
print(net[2].state_dict()) #_module, list

OrderedDict([('weight', tensor([[-0.1128, -0.3085, -0.1111,  0.0242,  0.1859, -0.1935, -0.0376, -0.2188]])), ('bias', tensor([-0.0815]))])


In [ ]:
# Directly vist
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data) #data是本身，用data是因为还可以访问梯度

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.0815], requires_grad=True)
tensor([-0.0815])


In [ ]:
net[2].weight.grad == None

True

**Visiting total parameters**

In [ ]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])   #ReLU:no parameters

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


***Remark:**

my_function(*[1, 2, 3]) == my_function(1, 2, 3)

In [ ]:
net.state_dict()['2.bias'].data #从list里面拿最后一层的bias的data

tensor([-0.0815])

In [ ]:
#从嵌套块收集params
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

# block2 嵌套4个block1
def block2():
  net = nn.Sequential()
  for i in range(4):
    net.add_module(f'block{i}', block1())
  return net

In [ ]:
rgnet = nn.Sequential(block2(), nn.Linear(4,1))
rgnet(X)

tensor([[0.0024],
        [0.0024]], grad_fn=<AddmmBackward0>)

In [ ]:
print(rgnet) #see the net

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [ ]:
# init params
def init_normal(m):
  if type(m) == nn.Linear:
    nn.init.normal_(m.weight, mean = 0, std = 0.01)
    nn.init.zeros_(m.bias)

In [ ]:
# apply(fn): 将一个函数fn递归地应用到模块自身以及该模块的每一个子模块(即在函数.children()中返回的子模块)
net.apply(init_normal) #For all layers in net, for all modules in layers, implement init_normal
net[0].weight.data[0], net[0].bias.data[0]


(tensor([-0.0009,  0.0118,  0.0016,  0.0097]), tensor(0.))

In [ ]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)

(tensor([1., 1., 1., 1.]), tensor(0.))

In [ ]:
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0] # API角度可玩，algorithm角度不要，weight全常数梯度会同

(tensor([1., 1., 1., 1.]), tensor(0.))

In [ ]:
def init_xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)#xavier初始化在梯度爆炸那一章提到过～
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) #宇宙无敌42hhh


In [ ]:
net[0].apply(init_xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([0.4043, 0.3027, 0.5847, 0.0599])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [ ]:
#如何深度自定义（瞎搞）
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        # if *[(a) for b in c()]后面没有[0]，结果是param+bias
        nn.init.uniform_(m.weight, -10, 10)
        # abs >= 5, return 1 ->  *1, no change
        m.weight.data *= m.weight.data.abs() >= 5


$$
 w \sim \begin{cases}
U(5,10) & \text{p=}\frac{1}{4}\\
0 & \text{p=}\frac{1}{2}\\
U(-10,-5) & \text{p=}\frac{1}{4}
\end{cases}
$$

In [ ]:
net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -8.0547, -5.9356,  0.0000],
        [ 5.8289,  9.4558, -0.0000, -6.3151]], grad_fn=<SliceBackward0>)

In [ ]:
#暴力法
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -6.0547, -3.9356,  2.0000])

**Share params**

In [ ]:
shared = nn.Linear(8,8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))

可以把shared想成是一个global variant？用同一个内存

In [ ]:
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## **自定义层**

layer也是`nn.Module`的subset

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn

In [ ]:
class CeneredLayer(nn.Module):
  def __init__(self):
     super().__init__()

  def forward(self, X):
    return X-X.mean()

In [ ]:
layer = CeneredLayer()
layer(torch.FloatTensor([1,2,3,4,5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [ ]:
# 自定义的层，和其他内置的一起玩
net = nn.Sequential(nn.Linear(8,128), CeneredLayer())

Y = net(torch.rand(4,8))
Y.mean()

tensor(-2.7940e-09, grad_fn=<MeanBackward0>)